In [11]:
import sys
sys.path.append('..')

from copy import deepcopy

import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as scheduler
from torch.utils.data import DataLoader
from torch.utils.data import random_split

from src.utils.data import read_and_prepare_metadata
from src.benchmarks.benchmark_factory import BenchmarkFactory
from src.scenarios.task_splitters import TaskSplitter
import src.models.torch_models as torch_models

In [12]:
DATASET_ROOT, IMG_SIZE = "../dataset", 512

In [13]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'

In [14]:
df_meta = read_and_prepare_metadata(DATASET_ROOT)
df_meta

,img_path,camera_type,pass_no,location,class,role,split,class_idx
0,train/DataSet_Xtion_RGB_train/000_Corridor1_RG...,Xtion,0,"(0.0, 0.0)",Corridor1,db,train,0
1,train/DataSet_Xtion_RGB_train/000_Corridor1_RG...,Xtion,0,"(0.0, 0.0)",Corridor1,db,train,0
2,train/DataSet_Xtion_RGB_train/001_Corridor1_RG...,Xtion,0,"(1.0, 1.0)",Corridor1,db,train,0
3,train/DataSet_Xtion_RGB_train/001_Corridor1_RG...,Xtion,0,"(1.0, 1.0)",Corridor1,db,train,0
4,train/DataSet_Xtion_RGB_train/002_Corridor1_RG...,Xtion,0,"(2.0, 2.0)",Corridor1,db,train,0
...,...,...,...,...,...,...,...,...
3301,test/DataSet_P40PRO_RGB_test3/078_F107_RGB_000...,P40PRO,3,"(878.0, 878.0)",F107,query,test,8
3302,test/DataSet_P40PRO_RGB_test3/079_F107_RGB_000...,P40PRO,3,"(879.0, 879.0)",F107,query,test,8
3303,test/DataSet_P40PRO_RGB_test3/081_F107_RGB_000...,P40PRO,3,"(881.0, 881.0)",F107,query,test,8
3304,test/DataSet_P40PRO_RGB_test3/082_F107_RGB_000...,P40PRO,3,"(882.0, 882.0)",F107,query,test,8


In [15]:
splitter = TaskSplitter(
    group_by="camera_type"
)
benchmark_factory = BenchmarkFactory(DATASET_ROOT, IMG_SIZE)

configs = splitter.split(df_meta)
benchmark = benchmark_factory.build_img_classification_benchmark(configs)

/Users/aszarata/informatyka/magisterka/agh-continual-learning-study/notebooks/../src/scenarios/task_splitters.py:39: UserWarning: you are shuffling a 'StringArray' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(unique_keys)


In [16]:
trainset_0 = benchmark.train_datasets_stream[0].dataset
valset_0 = benchmark.valid_datasets_stream[0].dataset
testset_0 = benchmark.test_datasets_stream[0].dataset


trainloader = DataLoader(trainset_0, batch_size=16, shuffle=True)
valloader = DataLoader(valset_0, batch_size=16, shuffle=True)
testloader = DataLoader(testset_0, batch_size=16, shuffle=True)

In [17]:
def check_balance(dataset, name):
    from collections import Counter
    labels = []
    for i in range(len(dataset)):
        _, label = dataset[i]
        labels.append(label)
    
    counts = Counter(labels)
    print(f"\n{name}:")
    for cls in sorted(counts):
        print(f"  Class {cls}: {counts[cls]} ({counts[cls]/len(labels):.1%})")
    
    imbalance = max(counts.values()) / min(counts.values())
    print(f"  Imbalance ratio: {imbalance:.1f}")

check_balance(trainset_0, "Train")
check_balance(valset_0, "Val")


Train:
  Class 0: 54 (12.1%)
  Class 1: 44 (9.9%)
  Class 2: 49 (11.0%)
  Class 3: 52 (11.7%)
  Class 4: 55 (12.3%)
  Class 5: 46 (10.3%)
  Class 6: 44 (9.9%)
  Class 7: 51 (11.4%)
  Class 8: 51 (11.4%)
  Imbalance ratio: 1.2

Val:
  Class 0: 3 (3.8%)
  Class 1: 12 (15.4%)
  Class 2: 5 (6.4%)
  Class 3: 10 (12.8%)
  Class 4: 7 (9.0%)
  Class 5: 14 (17.9%)
  Class 6: 7 (9.0%)
  Class 7: 11 (14.1%)
  Class 8: 9 (11.5%)
  Imbalance ratio: 4.7


In [18]:
model = torch_models.get_resnet34_for_cl(9, False)
criterion = nn.CrossEntropyLoss()
num_epochs = 40
early_stopping_patience = 5
scheduler_patience = 3
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [19]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.6,)
scheduler = scheduler.ReduceLROnPlateau(optimizer, patience=scheduler_patience)

In [20]:
steps_no_progress = 0
best_val_loss = 10**10

for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0.0
    with tqdm(trainloader, desc="Training") as pbar:
        for (X, y) in pbar:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()

            y_pred = model(X)
            loss = criterion(y_pred, y)

            loss.backward()
            optimizer.step()

            pbar.set_postfix(loss=loss.item())
            total_train_loss += loss.item()

    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        total_val_loss = 0.0
        with tqdm(valloader, desc="Validation") as pbar:
            for (X, y) in pbar:
                X, y = X.to(device), y.to(device)
                y_pred = model(X)
                
                loss = criterion(y_pred, y)
                pbar.set_postfix(loss=loss.item())
                total_val_loss += loss.item()

                _, predicted = torch.max(y_pred.data, 1)
                total += y.size(0)
                correct += (predicted == y).sum().item()

    train_loss = total_train_loss / len(trainloader)
    val_loss = total_val_loss / len(valloader)
    val_acc = 100 * correct / total

    steps_no_progress += 1

    if val_loss < best_val_loss:
        steps_no_progress = 0
        best_val_loss = val_loss
        best_model = deepcopy(model)

    scheduler.step(val_loss)

    print(f"Epoch: {epoch} | Train loss: {train_loss} | Val loss: {val_loss} | Val acc: {val_acc:.2f}%")
    if steps_no_progress >= early_stopping_patience:
        print("Early stop")
        break

Validation: 100%|██████████| 5/5 [00:00<00:00,  5.43it/s, loss=2.12]


Epoch: 0 | Train loss: 2.16718151313918 | Val loss: 2.4041367053985594 | Val acc: 3.85%


Validation: 100%|██████████| 5/5 [00:00<00:00,  5.48it/s, loss=9.45]


Epoch: 1 | Train loss: 1.599706185715539 | Val loss: 9.38684253692627 | Val acc: 12.82%


Validation: 100%|██████████| 5/5 [00:00<00:00,  5.52it/s, loss=3.43]


...

Epoch: 26 | Train loss: 0.35603555185454233 | Val loss: 0.4729055404663086 | Val acc: 84.62%


Validation: 100%|██████████| 5/5 [00:00<00:00,  5.37it/s, loss=0.372]


Epoch: 27 | Train loss: 0.33362487171377453 | Val loss: 0.4885295003652573 | Val acc: 84.62%


Validation: 100%|██████████| 5/5 [00:00<00:00,  5.48it/s, loss=0.563]

Epoch: 28 | Train loss: 0.2972405898783888 | Val loss: 0.4600228786468506 | Val acc: 84.62%
Early stop


In [23]:
torch.save(best_model.state_dict(), "../models/ResNet34-camera-task-0-2.pth")

In [ ]:
loaded_model = torch_models.get_resnet34_for_cl(9, False)
loaded_model.load_state_dict(torch.load( "../models/BEST-ResNet34-camera-task-0-2.pth"))
loaded_model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [25]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

loaded_model.eval()
all_preds = []
all_targets = []

with torch.no_grad():
    with tqdm(testloader, "Tests") as pbar:
        for X_test, y_test in pbar:
            X_test, y_test = X_test.to(device), y_test.to(device)
            output = loaded_model(X_test)
            pred = output.argmax(dim=1)
            all_preds.extend(pred.cpu().numpy())
            all_targets.extend(y_test.cpu().numpy())

cm = confusion_matrix(all_targets, all_preds)
print("Confusion Matrix:")
print(cm)

report = classification_report(all_targets, all_preds, digits=4)
print("\nClassification Report:")
print(report)

Tests: 100%|██████████| 9/9 [00:01<00:00,  5.65it/s]

Confusion Matrix:
[[ 7  0  0  0  0  0  0  0  0]
 [ 0 14  0  0  0  0  0  0  0]
 [ 4  0 12  0  0  1  0  0  0]
 [ 0  0  0  9  0  0  0  0  0]
 [ 1  0  0  0 14  2  0  0  1]
 [ 0  0  0  0  0 18  0  0  1]
 [ 0  0  0  0  1  0  9  5  0]
 [ 0  0  0  0  0  0  1 14  3]
 [ 2  0  0  0  0  1  0  1 10]]

Classification Report:
              precision    recall  f1-score   support

           0     0.5000    1.0000    0.6667         7
           1     1.0000    1.0000    1.0000        14
           2     1.0000    0.7059    0.8276        17
           3     1.0000    1.0000    1.0000         9
           4     0.9333    0.7778    0.8485        18
           5     0.8182    0.9474    0.8780        19
           6     0.9000    0.6000    0.7200        15
           7     0.7000    0.7778    0.7368        18
           8     0.6667    0.7143    0.6897        14

    accuracy                         0.8168       131
   macro avg     0.8354    0.8359    0.8186       131
weighted avg     0.8495    0.8168    